<a href="https://colab.research.google.com/github/kodohoi/aidemy_git/blob/master/RnadomAffinePartial_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchio as tio
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import random
from ctviewer import CTViewer

In [4]:
images_dir = dataset_dir / 'image'
labels_dir = dataset_dir / 'label'
image_paths = sorted(images_dir.glob('*.nii.gz'))
label_paths = sorted(labels_dir.glob('*.nii.gz'))
assert len(image_paths) == len(label_paths)

subjects = []
for (image_path, label_path) in zip(image_paths, label_paths):
    subject = tio.Subject(
        mri=tio.ScalarImage(image_path),
        brain=tio.LabelMap(label_path),
    )
    subjects.append(subject)
dataset = tio.SubjectsDataset(subjects)
print('Dataset size:', len(dataset), 'subjects')

Dataset size: 566 subjects


In [7]:
subject = tio.Subject({"image": tio.ScalarImage("/012c12fe09c3.dcm")})

In [15]:
subject.image

RuntimeError: ignored

In [12]:
import numpy as np
import torch
import torchio as tio

class RandomAffinePartial:
    def __init__(self, degrees=10.0, translation=0.1, scales=0.1, patch_size = (10,10,10), p=0.5):
        self.degrees = degrees
        self.translation = translation
        self.scales = scales
        self.p = p
        self.patch_size = patch_size
        
    def __call__(self, image):
        if random.random() < self.p:
            # Convert patch_size to numpy array
            patch_size = np.array(self.patch_size)

            # Get the spatial dimensions of the image
            _, si, sj, sk = image.shape

            # Select a random location for the patch
            patch_start = np.array([
                np.random.randint(0, si - patch_size[0] + 1),
                np.random.randint(0, sj - patch_size[1] + 1),
                np.random.randint(0, sk - patch_size[2] + 1)
            ])

            # Extract the patch
            patch = image.data[:,patch_start[0]:patch_start[0] + patch_size[0], patch_start[1]:patch_start[1] + patch_size[1], patch_start[2]:patch_start[2] + patch_size[2]]
            patch_image = tio.ScalarImage(tensor=patch)

            # Create a random affine transform
            transform = tio.RandomAffine(
                degrees=self.degrees,
                translation=self.translation,
                scales=self.scales
            )

            # Apply the transform to the patch
            transformed_patch = transform(patch_image).data.short()

            # Create a new image with the transformed patch at the same location
            transformed_image_data = image.data.clone()
            transformed_image_data[:,patch_start[0]:patch_start[0] + patch_size[0], patch_start[1]:patch_start[1] + patch_size[1], patch_start[2]:patch_start[2] + patch_size[2]] = transformed_patch
            transformed_image = tio.ScalarImage(tensor=transformed_image_data)

            return transformed_image


In [13]:
process = tio.Compose([RandomAffinePartial(degrees=45, translation=5, scales=0.0, p=1.0)])

In [14]:
processed_image= process(subjects[0].mri.data)

In [17]:
processed_image.shape

(1, 83, 44, 55)